In [4]:
import numpy as np
import pandas as pd
from scipy import signal
from scipy.signal import butter, filtfilt, freqz, welch


In [2]:
all_data = np.load('processed_data/all_data.npy')

In [ ]:
def highpassFilter(data, sr, threshold):
    b,a = butter(2, threshold/(sr/2),'high')
    return filtfilt(b,a,data)

In [3]:
def lowpassFilter(data, sr, threshold):
    b,a = butter(2, threshold/(sr/2),'low')
    return filtfilt(b,a,data)

(270, 1380, 5)

In [ ]:
def WL(data):
    wl = np.sum(np.abs(np.diff(data)))
    return wl / len(data)

In [ ]:
def MAV(data):
    return np.sum(np.abs(data))/len(data)

In [ ]:
def SSC(data,threshold):
    res = 0
    for i in range(1, len(data)-1):
        curr = (data[i]-data[i-1]) * (data[i+1]-data[i])
        if curr >= threshold:
            res += 1
    return res

In [ ]:
def ZC(data):
    """
    Counts how many times the signal crosses zero. 
    """
    res = 0
    for i in range(1, len(data)):
        curr = data[i] * data[i-1]
        if curr < 0:
            res += 1
    return res

In [ ]:
def VAR(data):
    """
    Measures the spread of signal values around the mean. It reflects the level of muscle activation.
    """
    return np.var(data)

In [ ]:
def RMS(data):
    """
    Measures the amplitude of the signal.
    """
    return np.sqrt(np.mean(data**2))

In [ ]:
def PSD(data,sr):
    f, pxx = welch(data, sr, nperseg=1024) 